# Skript um die diskretisierte DGL für die Massenerhaltung umzuformen


In [2]:
from sympy import symbols, expand, pprint, init_printing, latex, simplify, collect, Eq, IndexedBase
init_printing()  # nice pretty/LaTeX printing in notebooks

# Indizes & Parameter
i = symbols('i', integer=True)     # der Laufindex
delta_r = symbols('delta_r', positive=True)
r = IndexedBase('r')

# Felder (als diskrete Gitterwerte):
k_eff   = IndexedBase('k_eff')
rho_a = IndexedBase('rho_a')
T_f   = IndexedBase('T_f')
w_f   = IndexedBase('w_f')
C   = symbols('C')
i_sv   = symbols('i_sv')    # Skalar
w_sat_i = symbols('w_sat_i')
Alpha, Beta, Gamma = symbols('Alpha, Beta, Gamma')

eq = Eq(
    (k_eff[i]/r[i]) * ( (T_f[i+1] - T_f[i-1])/(2*delta_r)
    + r[i] * (T_f[i+1] - 2*T_f[i] - T_f[i-1])/(delta_r**2) ),
    -i_sv*C*rho_a[i]*(w_f[i] - w_sat_i)
)

print('\nEingegeben Gleichung:\n')
pprint(eq)

# Alles auf eine Seite bringen -> Ausdruck
expr = expand(eq.lhs - eq.rhs)

# Nach w_m, w_i, w_n gruppieren
col = collect(expr, [T_f[i+1], T_f[i], T_f[i-1]], evaluate=False)
alpha = simplify(col.get(T_f[i+1], 0))
beta = simplify(col.get(T_f[i],   0))
gamma = simplify(col.get(T_f[i-1], 0))
const = simplify(col.get(1, 0))     # Restterm ohne w_m, w_i, w_n

# Rekonstruktion der linken Seite in gewünschter Form
lhs_grouped = T_f[i+1]*alpha + T_f[i]*beta + T_f[i-1]*gamma + const

# Als Gleichung (== 0) oder wieder nach rechts umstellen:
eq_grouped_zero = Eq(lhs_grouped, 0)
eq_grouped      = Eq(T_f[i+1]*alpha + T_f[i]*beta + T_f[i-1]*gamma, -const)

print('\nGruppiert für Matrix Notation:\n')
pprint(eq_grouped)

print('\nFaktoren:\n')
pprint(Eq(Alpha, alpha))
pprint(Eq(Beta, beta))
pprint(Eq(Gamma, gamma))

print('\nLaTeX:\n')
print(latex(eq_grouped))


Eingegeben Gleichung:

⎛T_f[i + 1] - T_f[i - 1]   (T_f[i + 1] - T_f[i - 1] - 2⋅T_f[i])⋅r[i]⎞          ↪
⎜─────────────────────── + ─────────────────────────────────────────⎟⋅k_eff[i] ↪
⎜         2⋅δᵣ                                  2                   ⎟          ↪
⎝                                             δᵣ                    ⎠          ↪
────────────────────────────────────────────────────────────────────────────── ↪
                                     r[i]                                      ↪

↪                                      
↪                                      
↪                                      
↪                                      
↪  = -C⋅iₛᵥ⋅(-wₛₐₜ ᵢ + w_f[i])⋅rho_a[i]
↪                                      

Gruppiert für Matrix Notation:

⎛  k_eff[i]    k_eff[i]⎞              ⎛k_eff[i]    k_eff[i]⎞              2⋅T_ ↪
⎜- ───────── - ────────⎟⋅T_f[i - 1] + ⎜───────── + ────────⎟⋅T_f[i + 1] - ──── ↪
⎜  2⋅δᵣ⋅r[i]       2   ⎟              ⎜2⋅δᵣ⋅r[i]      